In [1]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
#environnement Frozen Lake -- mettre v0 ou v1 suivant votre version
env_fl = gym.make('FrozenLake-v1')

## Environnement maison : exemple du personnage
On reprend l'API de gym avec les méthodes reset et step

In [3]:
class Personnage_Env:
    """environnement type API gym représentant l'exemple du cours"""
    
    def __init__(self, R):
        """
        R : paramètre de récompense de l'action 1:Récolter dans l'état 1:Affamé
        """
        
        self.observation_space = gym.spaces.Discrete(3) # 0 :OK, 1 :Affamé, 2:Mort
        self.action_space = gym.spaces.Discrete(2)      # 0 : Manger 1 : Récolter
        self.P = {0: {0 : [(0.8, 0, 1.0, False),
                          (0.2, 1, 1.0, False )],
                     1 : [(0.5, 0, 10.0, False),
                         (0.5, 1, 10.0, False)]},
                 1 : {0: [(0.5, 0, 0.0, False),
                         (0.25, 1, 0.0, False),
                         (0.25, 2, 0.0, True)],
                     1 : [(0.5, 1, R, False),
                         (0.5, 2, R, True)]},
                 2 : {0 : [(1.0, 2, 0.0, True)],
                     1 : [(1.0, 2, 0.0, True)]}}
        self.reset()
        
    def reset(self):
        self.current_state = 0
        return self.current_state
        
    def step(self, action):
        transitions = self.P[self.current_state][action]
        
        #indices des actions possibles
        choices = list(range(len(transitions)))
        
        # probas correspondantes
        probas = [t[0] for t in transitions]
        
        #choisit i dans choices suivant la proba 
        i = np.random.choice(choices, 1, p=probas)[0]
        
        #mise à jour
        self.current_state = choices[i]
        return transitions[i][1], transitions[i][2], transitions[i][3],  None

In [5]:
env_p = Personnage_Env( R = 5.0)

## Evaluation de la politique aléatoire uniforme

In [4]:
def value_iteration_uniform_policy(env, nb_iter):
    """
    iteration de valeurs sur les états pour évaluer la politique uniforme
    """
    
    states = list(range(env.observation_space.n))
    actions = list(range(env.action_space.n))
    nba = len(actions)

    state_values = defaultdict(lambda :0.0)
    
    for _ in range(nb_iter):
        for s0 in states:
            # pour s0 état et a0 action : env.P[s0][a0] est une liste de transitions t
            # t[0] est la proba d'aller vers un état t[1] avec un reward t[2]
            state_values[s0] = sum([ sum([ t[0]/ nba * (t[2] + state_values[t[1]]) for t in env.P[s0][a0]])   for a0 in env.P[s0] ])
                
    return state_values

In [7]:
value_iteration_uniform_policy(env_p, 200)

defaultdict(<function __main__.value_iteration_uniform_policy.<locals>.<lambda>()>,
            {0: 32.85714285714285, 1: 17.14285714285714, 2: 0.0})

In [8]:
value_iteration_uniform_policy(env_fl, 200)

defaultdict(<function __main__.value_iteration_uniform_policy.<locals>.<lambda>()>,
            {0: 0.013939796242315783,
             4: 0.01624866518514242,
             1: 0.011630927299489151,
             5: 0.0,
             2: 0.02095298565615168,
             6: 0.04075153684089006,
             3: 0.010476492828075838,
             7: 0.0,
             8: 0.034806199313111484,
             10: 0.14205316170740856,
             12: 0.0,
             9: 0.08816993275419203,
             13: 0.17582036999624806,
             14: 0.43929117723455224,
             11: 0.0,
             15: 0.0})